In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
from mako.template import Template
import requests
from requests.compat import quote
from PIL import Image, ImageFile
from openpyxl import Workbook

import calendar
from io import BytesIO
import re

from modules.wmtools import is_commons_file, get_hash, sanitize_filename

In [ ]:
def get_image_dimensions (uri, headers) :
    image_url = quote(uri.encode('utf-8'), ':/')
    try: 
        r = requests.get(image_url, headers=headers)
        image = Image.open(BytesIO(r.content))
        return image.size[0] * image.size[1]
    except :
        print ('Exception')
        return 0

In [ ]:
commons_site = pb.Site("commons", "commons")
wb = Workbook()
ws = wb.active
EXCEL_FILE = "diario.madrid.es.xlsx"

In [ ]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [ ]:
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'
headers = {'User-Agent': user_agent}
url_base = 'https://diario.madrid.es/blog/notas-de-prensa/'

r = requests.get(url_base, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
pages = soup.find_all("a", class_="page-numbers")
final_page = int(pages[-2].get_text())-1
final_page

In [ ]:
page_number = 100
row = []
for i in range (final_page) :
    if i > page_number :
        break

    if i == 0 :
        url = url_base
    else :
        url = '{0}page/{1}'.format(url_base, i+1)
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    articles = soup.find_all("h2", class_="post-title")
    for article in articles :
        article_url = article.a["href"]
        print('-----')
        print(article_url)
        r = requests.get(article_url, headers=headers)
        soup = BeautifulSoup(r.text, 'html.parser')

        title = soup.find_all("h1", class_="post-title")[0].get_text().strip().replace('  ', ' ')
        print (title)
        date = '-'.join(soup.find_all("div", class_="post-date")[0].get_text().strip().split('/')[::-1])
        year = date.split('-')[0]
        month = calendar.month_name[int(date.split('-')[1])]
        body = soup.find_all("div", class_="post-content")
        p_description = body[0].find_all("p")
        #print (p_description)
        for p in p_description :
            if len(p.get_text()) > 10:
                description = p.get_text()
                break

        counter = 0
        images = [image.a["href"] for image in soup.find_all("div", class_="gallery-icon")]
        for image in images:
            try:
                image_url = quote(image.encode('utf-8'), ':/')
            except :
                continue
            print (image_url)
            image_name = sanitize_filename('{} {:02d}.jpg'.format(title, counter+1))
            image_path = os.path.join(images_directory, image_name)
            print (image_path)
            try:
                r = requests.get(image_url, headers=headers)
                image = Image.open(BytesIO(r.content))
                image.save(image_path)
                row = [image_path, image_url, title, article_url, date, year, month, description]
                ws.append(row)
                pass
            except :
                print ('Failed download. Skipping')
            counter += 1

        images = soup.find_all(attrs={'class': re.compile(r"wp-image-\d*")})
        for i, image in enumerate(images) :
            try:
                links = {int(x.strip().split(' ')[1][:-1]): x.strip().split(' ')[0] for x in image.get('srcset').split(',')}
                sortedlist = [(k, links[k]) for k in sorted(links)]
                link = sortedlist[-1][1]
            except :
                link = image.get('src')
                
            if image.parent.get('href') == None :
                # No parent
                print ('No parent')
                image_url = link
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            elif get_image_dimensions(image.parent.get('href'), headers) > get_image_dimensions(link, headers) :
                # Bigger size
                print ('Bigger size')
                image_url = image.parent.get('href')
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            else :
                # Others
                print ('Other')
                image_url = link
                image_url = quote(image_url.encode('utf-8'), ':/')
                print (image_url)
            image_name = sanitize_filename('{} {:02d}.jpg'.format(title, counter+1))
            image_path = os.path.join(images_directory, image_name)
            print (image_path)

            #print (image_path)
            try:
                r = requests.get(image_url, headers=headers)
                image = Image.open(BytesIO(r.content))
                image.save(image_path)
                row = [image_path, image_url, title, article_url, date, year, month, description]
                ws.append(row)
                pass
            except :
                print ('Failed download. Skipping')
            counter += 1

In [ ]:
wb.save(os.path.join(cwd, EXCEL_FILE))